# Transformer
[参考链接](https://zhuanlan.zhihu.com/p/54356280)


[gogle tensor2tensor 上面Transforme的例子，带动态图的](https://colab.research.google.com/github/tensorflow/tensor2tensor/blob/master/tensor2tensor/notebooks/hello_t2t.ipynb#scrollTo=0a69r1KDiZDe)

# 原理
**从宏观的视角开始**



首先将这个模型看成是一个黑箱操作。在机器翻译中，就是输入一种语言，输出另一种语言。

![](https://pic1.zhimg.com/80/v2-c85c8c8ec423d6c9333e313adcee4934_hd.jpg)

那么拆开这个黑箱，我们可以看到它是由编码组件、解码组件和它们之间的连接组成。



![](https://pic1.zhimg.com/80/v2-1706d9c0984be75b9017b425fcdc9784_hd.jpg)

编码组件部分由一堆编码器（encoder）构成（论文中是将6个编码器叠在一起——数字6没有什么神奇之处，你也可以尝试其他数字）。解码组件部分也是由相同数量（与编码器对应）的解码器（decoder）组成的。

所有的编码器在结构上都是相同的，但它们没有共享参数。每个编码器都可以分解成两个子层。

![](https://pic2.zhimg.com/80/v2-fbb5dbc286b9f9cec2ddbc5eae2bf5a9_hd.jpg)

从编码器输入的句子首先会经过一个自注意力（self-attention）层，这层帮助编码器在对每个单词编码时关注输入句子的其他单词。我们将在稍后的文章中更深入地研究自注意力。



自注意力层的输出会传递到前馈（feed-forward）神经网络中。每个位置的单词对应的前馈神经网络都完全一样（译注：另一种解读就是一层窗口为一个单词的一维卷积神经网络）。



解码器中也有编码器的自注意力（self-attention）层和前馈（feed-forward）层。除此之外，这两个层之间还有一个注意力层，用来关注输入句子的相关部分（和seq2seq模型的注意力作用相似）。

将张量引入图景



我们已经了解了模型的主要部分，接下来我们看一下各种向量或张量（译注：张量概念是矢量概念的推广，可以简单理解矢量是一阶张量、矩阵是二阶张量。）是怎样在模型的不同部分中，将输入转化为输出的。



像大部分NLP应用一样，我们首先将每个输入单词通过词嵌入算法转换为词向量。

![](https://pic2.zhimg.com/80/v2-1f54ae99e2edaa4471a0f0f111e6dea5_hd.png)

每个单词都被嵌入为512维的向量，我们用这些简单的方框来表示这些向量。



词嵌入过程只发生在最底层的编码器中。所有的编码器都有一个相同的特点，即它们接收一个向量列表，列表中的每个向量大小为512维。在底层（最开始）编码器中它就是词向量，但是在其他编码器中，它就是下一层编码器的输出（也是一个向量列表）。向量列表大小是我们可以设置的超参数——一般是我们训练集中最长句子的长度。



将输入序列进行词嵌入之后，每个单词都会流经编码器中的两个子层。

![](https://pic2.zhimg.com/80/v2-d7b0bb93c9f7e7185d690b6df83d8859_hd.jpg)

接下来我们看看Transformer的一个核心特性，在这里输入序列中每个位置的单词都有自己独特的路径流入编码器。在自注意力层中，这些路径之间存在依赖关系。而前馈（feed-forward）层没有这些依赖关系。因此在前馈（feed-forward）层时可以并行执行各种路径。



然后我们将以一个更短的句子为例，看看编码器的每个子层中发生了什么。



现在我们开始“编码”



如上述已经提到的，一个编码器接收向量列表作为输入，接着将向量列表中的向量传递到自注意力层进行处理，然后传递到前馈神经网络层中，将输出结果传递到下一个编码器中。

![](https://pic2.zhimg.com/80/v2-7173f8fa4d601a5255af46d48e9d370d_hd.jpg)

输入序列的每个单词都经过自编码过程。然后，他们各自通过前向传播神经网络——完全相同的网络，而每个向量都分别通过它。

从宏观视角看自注意力机制



不要被我用自注意力这个词弄迷糊了，好像每个人都应该熟悉这个概念。其实我之也没有见过这个概念，直到读到Attention is All You Need 这篇论文时才恍然大悟。让我们精炼一下它的工作原理。



例如，下列句子是我们想要翻译的输入句子：

The animal didn't cross the street because it was too tired


这个“it”在这个句子是指什么呢？它指的是street还是这个animal呢？这对于人类来说是一个简单的问题，但是对于算法则不是。



当模型处理这个单词“it”的时候，自注意力机制会允许“it”与“animal”建立联系。



随着模型处理输入序列的每个单词，自注意力会关注整个输入序列的所有单词，帮助模型对本单词更好地进行编码。



如果你熟悉RNN（循环神经网络），回忆一下它是如何维持隐藏层的。RNN会将它已经处理过的前面的所有单词/向量的表示与它正在处理的当前单词/向量结合起来。而自注意力机制会将所有相关单词的理解融入到我们正在处理的单词中。



![](https://pic1.zhimg.com/80/v2-8b18b40210d7d9a37bb35e3d960d78c0_hd.jpg)

当我们在编码器#5（栈中最上层编码器）中编码“it”这个单词的时，注意力机制的部分会去关注“The Animal”，将它的表示的一部分编入“it”的编码中。



请务必检查Tensor2Tensor notebook ，在里面你可以下载一个Transformer模型，并用交互式可视化的方式来检验。

**从微观视角看自注意力机制**



首先我们了解一下如何使用向量来计算自注意力，然后来看它实怎样用矩阵来实现。



计算自注意力的第一步就是从每个编码器的输入向量（每个单词的词向量）中生成三个向量。也就是说对于每个单词，我们创造一个查询向量、一个键向量和一个值向量。这三个向量是通过词嵌入与三个权重矩阵后相乘创建的。



可以发现这些新向量在维度上比词嵌入向量更低。他们的维度是64，而词嵌入和编码器的输入/输出向量的维度是512. 但实际上不强求维度更小，这只是一种基于架构上的选择，它可以使多头注意力（multiheaded attention）的大部分计算保持不变。

![](https://pic2.zhimg.com/80/v2-bac717483cbeb04d1b5ef393eb87a16d_hd.jpg)

X1与WQ权重矩阵相乘得到q1, 就是与这个单词相关的查询向量。最终使得输入序列的每个单词的创建一个查询向量、一个键向量和一个值向量。



什么是查询向量、键向量和值向量向量？



它们都是有助于计算和理解注意力机制的抽象概念。请继续阅读下文的内容，你就会知道每个向量在计算注意力机制中到底扮演什么样的角色。



计算自注意力的第二步是计算得分。假设我们在为这个例子中的第一个词“Thinking”计算自注意力向量，我们需要拿输入句子中的每个单词对“Thinking”打分。这些分数决定了在编码单词“Thinking”的过程中有多重视句子的其它部分。



这些分数是通过打分单词（所有输入句子的单词）的键向量与“Thinking”的查询向量相点积来计算的。所以如果我们是处理位置最靠前的词的自注意力的话，第一个分数是q1和k1的点积，第二个分数是q1和k2的点积。

![](https://pic2.zhimg.com/80/v2-373fb39e650fa85976bbb6eaf67b31ed_hd.jpg)

第三步和第四步是将分数除以8(8是论文中使用的键向量的维数64的平方根，这会让梯度更稳定。这里也可以使用其它值，8只是默认值)，然后通过softmax传递结果。softmax的作用是使所有单词的分数归一化，得到的分数都是正值且和为1。


![](https://pic2.zhimg.com/80/v2-5591c2b55d9e31e744f884bacf959b45_hd.jpg)

这个softmax分数决定了每个单词对编码当下位置（“Thinking”）的贡献。显然，已经在这个位置上的单词将获得最高的softmax分数，但有时关注另一个与当前单词相关的单词也会有帮助。



第五步是将每个值向量乘以softmax分数(这是为了准备之后将它们求和)。这里的直觉是希望关注语义上相关的单词，并弱化不相关的单词(例如，让它们乘以0.001这样的小数)。



第六步是对加权值向量求和（译注：自注意力的另一种解释就是在编码某个单词时，就是将所有单词的表示（值向量）进行加权求和，而权重是通过该词的表示（键向量）与被编码词表示（查询向量）的点积并通过softmax得到。），然后即得到自注意力层在该位置的输出(在我们的例子中是对于第一个单词)。

![](https://pic4.zhimg.com/80/v2-609de8f8f8e628e6a9ca918230c70d67_hd.jpg)


这样自自注意力的计算就完成了。得到的向量就可以传给前馈神经网络。然而实际中，这些计算是以矩阵形式完成的，以便算得更快。那我们接下来就看看如何用矩阵实现的。

**通过矩阵运算实现自注意力机制**



第一步是计算查询矩阵、键矩阵和值矩阵。为此，我们将将输入句子的词嵌入装进矩阵X中，将其乘以我们训练的权重矩阵(WQ，WK，WV)。

![](https://pic3.zhimg.com/80/v2-ca0c37cadf0a817e4836aa5a985bf1b6_hd.jpg)

x矩阵中的每一行对应于输入句子中的一个单词。我们再次看到词嵌入向量 (512，或图中的4个格子)和q/k/v向量(64，或图中的3个格子)的大小差异。



最后，由于我们处理的是矩阵，我们可以将步骤2到步骤6合并为一个公式来计算自注意力层的输出。

1[](https://pic3.zhimg.com/80/v2-7a954ddb05bee33c7c8a9c7a99fe7b6e_hd.jpg)

自注意力的矩阵运算形式

**“大战多头怪”**



通过增加一种叫做“多头”注意力（“multi-headed” attention）的机制，论文进一步完善了自注意力层，并在两方面提高了注意力层的性能：



1.它扩展了模型专注于不同位置的能力。在上面的例子中，虽然每个编码都在z1中有或多或少的体现，但是它可能被实际的单词本身所支配。如果我们翻译一个句子，比如“The animal didn’t cross the street because it was too tired”，我们会想知道“it”指的是哪个词，这时模型的“多头”注意机制会起到作用。



2.它给出了注意力层的多个“表示子空间”（representation subspaces）。接下来我们将看到，对于“多头”注意机制，我们有多个查询/键/值权重矩阵集(Transformer使用八个注意力头，因此我们对于每个编码器/解码器有八个矩阵集合)。这些集合中的每一个都是随机初始化的，在训练之后，每个集合都被用来将输入词嵌入(或来自较低编码器/解码器的向量)投影到不同的表示子空间中。

![](https://pic2.zhimg.com/80/v2-6214664f4f8d080ab41e6909f15b1f69_hd.jpg)

在“多头”注意机制下，我们为每个头保持独立的查询/键/值权重矩阵，从而产生不同的查询/键/值矩阵。和之前一样，我们拿X乘以WQ/WK/WV矩阵来产生查询/键/值矩阵。



如果我们做与上述相同的自注意力计算，只需八次不同的权重矩阵运算，我们就会得到八个不同的Z矩阵。

![](https://pic4.zhimg.com/80/v2-9adff6a99688705389e3d96bc0a72d2f_hd.jpg)

这给我们带来了一点挑战。前馈层不需要8个矩阵，它只需要一个矩阵(由每一个单词的表示向量组成)。所以我们需要一种方法把这八个矩阵压缩成一个矩阵。那该怎么做？其实可以直接把这些矩阵拼接在一起，然后用一个附加的权重矩阵WO与它们相乘。

![](https://pic2.zhimg.com/80/v2-745f36fdfed78f28629bbdd4bac8660d_hd.jpg)

这几乎就是多头自注意力的全部。这确实有好多矩阵，我们试着把它们集中在一个图片中，这样可以一眼看清。

![](https://pic1.zhimg.com/80/v2-eecbb4c1d2575844f9fb37cc144bc94c_hd.jpg)

既然我们已经摸到了注意力机制的这么多“头”，那么让我们重温之前的例子，看看我们在例句中编码“it”一词时，不同的注意力“头”集中在哪里：

![](https://pic3.zhimg.com/80/v2-f268e009970206d27c6762fd634c2d12_hd.jpg)

当我们编码“it”一词时，一个注意力头集中在“animal”上，而另一个则集中在“tired”上，从某种意义上说，模型对“it”一词的表达在某种程度上是“animal”和“tired”的代表。



然而，如果我们把所有的attention都加到图示里，事情就更难解释了：

![](https://pic4.zhimg.com/80/v2-fc5f733a4f5dfc654f1211ba25eb4797_hd.jpg)

**使用位置编码表示序列的顺序**



到目前为止，我们对模型的描述缺少了一种理解输入单词顺序的方法。



为了解决这个问题，Transformer为每个输入的词嵌入添加了一个向量。这些向量遵循模型学习到的特定模式，这有助于确定每个单词的位置，或序列中不同单词之间的距离。这里的直觉是，将位置向量添加到词嵌入中使得它们在接下来的运算中，能够更好地表达的词与词之间的距离。

![](https://pic4.zhimg.com/80/v2-ac9c8d239071146e24db823e2ae4f43b_hd.jpg)
为了让模型理解单词的顺序，我们添加了位置编码向量，这些向量的值遵循特定的模式。



如果我们假设词嵌入的维数为4，则实际的位置编码如下：

![](https://pic1.zhimg.com/80/v2-437b1a89c29368d6e821c3c0be001690_hd.jpg)

尺寸为4的迷你词嵌入位置编码实例



这个模式会是什么样子？



在下图中，每一行对应一个词向量的位置编码，所以第一行对应着输入序列的第一个词。每行包含512个值，每个值介于1和-1之间。我们已经对它们进行了颜色编码，所以图案是可见的。


![](https://pic1.zhimg.com/80/v2-2d1e32c22c7e4de44d731e4969e27f08_hd.jpg)


20字(行)的位置编码实例，词嵌入大小为512(列)。你可以看到它从中间分裂成两半。这是因为左半部分的值由一个函数(使用正弦)生成，而右半部分由另一个函数(使用余弦)生成。然后将它们拼在一起而得到每一个位置编码向量。



原始论文里描述了位置编码的公式(第3.5节)。你可以在 get_timing_signal_1d()中看到生成位置编码的代码。这不是唯一可能的位置编码方法。然而，它的优点是能够扩展到未知的序列长度(例如，当我们训练出的模型需要翻译远比训练集里的句子更长的句子时)。



残差模块



在继续进行下去之前，我们需要提到一个编码器架构中的细节：在每个编码器中的每个子层（自注意力、前馈网络）的周围都有一个残差连接，并且都跟随着一个“层-归一化”步骤。



层-归一化步骤：

https://arxiv.org/abs/1607.06450

![](https://pic2.zhimg.com/80/v2-28b65db48ce0b5fb57f0fd954b13a169_hd.jpg)

如果我们去可视化这些向量以及这个和自注意力相关联的层-归一化操作，那么看起来就像下面这张图描述一样：



![](https://pic4.zhimg.com/80/v2-f8c32c325da61ee587abcd2426854b33_hd.jpg)